<a href="https://www.kaggle.com/code/mattop/world-series-ratings-data-scrape?scriptVersionId=114530234" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import re

In [2]:
df0 = pd.read_html("https://en.wikipedia.org/wiki/World_Series_television_ratings")

In [3]:
df = df0[5]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Year     57 non-null     object
 1   Network  57 non-null     object
 2   Results  57 non-null     object
 3   Average  57 non-null     object
 4   Game 1   56 non-null     object
 5   Game 2   56 non-null     object
 6   Game 3   56 non-null     object
 7   Game 4   56 non-null     object
 8   Game 5   56 non-null     object
 9   Game 6   56 non-null     object
 10  Game 7   57 non-null     object
dtypes: object(11)
memory usage: 5.0+ KB


In [5]:
df.head(3)

,Year,Network,Results,Average,Game 1,Game 2,Game 3,Game 4,Game 5,Game 6,Game 7
0,2022,Fox,"Houston Astros 4, Philadelphia Phillies 2",6.108(11.762 Mviewers),5.69(11.475 Mviewers)[19],5.34(10.789 Mviewers)[20],6.11(11.162 Mviewers)[21],6.48(11.809 Mviewers),6.92(12.786 Mviewers),6.11(12.549 M viewers),No Game
1,2021,Fox,"Atlanta Braves 4, Houston Astros 2",6.5/16(11.744 Mviewers),6.1(10.811 M viewers)[22],5.8(10.280 M viewers)[23],6.1(11.232 M viewers)[24],5.65(10.511 M viewers)[25],7.4(13.644 M viewers)[25],7.9(13.986 M viewers)[26],No Game
2,2020,Fox,"Los Angeles Dodgers 4,Tampa Bay Rays 2",5.2/12(9.785 Mviewers)[6],5.1/11(9.195 Mviewers)[27],5.0/11(8.950 Mviewers)[28],4.3/10(8.156 Mviewers)[11],4.8/12(9.332 Mviewers)[29],5.3/11(10.059 Mviewers)[30],6.8/15(12.627 Mviewers)[6],No Game


In [6]:
def extract_int(string):
    return int(re.search(r'\d+', string).group())


def extract_str(string):
    return re.search(r'\D+', string).group()


def extract_in_parens(string):
    '''use a regular expression to extract the characters within parentheses'''
    try:
        values = re.search(r'\((.+)\)', string).group(1)
        
        if values.split()[0] == "naviewers": return float("nan")
        
        return values.split()[0]
    
    except:
        return float("nan")
    
    
def pad_with_zeros(string, length):
    '''add zeros to the end of string'''
    try:        
        padded_string = string + '0' * length
        
        return padded_string
    
    except:
        return float("nan")

In [7]:
df = df.drop(df.index[-1])
df = df.drop(index = 29)

df[["Winning Team", "Losing Team"]] = df["Results"].str.split(",", 1, expand = True)

df.insert(loc = 12, column = "Winning Team Wins", value = df["Winning Team"].apply(extract_int))

df["Losing Team Wins"] = df["Losing Team"].apply(extract_int)

df.insert(loc = 11, column = "Total Games Played",
          value = df["Winning Team Wins"] + df["Losing Team Wins"])

df = df.drop(columns = "Winning Team Wins")

df["Winning Team"] = df["Winning Team"].apply(extract_str)
df["Losing Team"] = df["Losing Team"].apply(extract_str)

df = df.drop(columns = "Results")
    
df = df.drop(df.index[-1])

df["Game 7"][37] = 55_000_000

for i in df.columns[2:10]:
    df[i] = df[i].str.replace(".", "")
    df[i] = df[i].apply(extract_in_parens)
    df[i] = df[i].apply(pad_with_zeros, length = 3)
    df[i] = df[i].astype(float).astype("Int32")
    
df["Year"] = df["Year"].astype(int)

# compute average viewership of missing average values
for i in range(51, 55):
    
    if i < 53: df["Average"][i] = ((df["Game 1"][i] + df["Game 2"][i] + df["Game 3"][i] + df["Game 4"][i] \
                                  + df["Game 5"][i] + df["Game 6"][i] + df["Game 7"][i]) / 7).astype(int)
    
    else: df["Average"][i] = ((df["Game 1"][i] + df["Game 2"][i] + df["Game 3"][i] \
                             + df["Game 4"][i] + df["Game 5"][i]) / 5).astype(int)

df["Game 7"][37] = 55_000_000

df.columns = ["year", "network", "average_audience", "game_1_audience", "game_2_audience", "game_3_audience",
              "game_4_audience", "game_5_audience", "game_6_audience", "game_7_audience", "total_games_played",
              "winning_team", "losing_team", "losing_team_wins"]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 54
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   year                54 non-null     int64 
 1   network             54 non-null     object
 2   average_audience    54 non-null     Int32 
 3   game_1_audience     53 non-null     Int32 
 4   game_2_audience     52 non-null     Int32 
 5   game_3_audience     53 non-null     Int32 
 6   game_4_audience     53 non-null     Int32 
 7   game_5_audience     44 non-null     Int32 
 8   game_6_audience     31 non-null     Int32 
 9   game_7_audience     18 non-null     Int32 
 10  total_games_played  54 non-null     int64 
 11  winning_team        54 non-null     object
 12  losing_team         54 non-null     object
 13  losing_team_wins    54 non-null     int64 
dtypes: Int32(8), int64(3), object(3)
memory usage: 7.1+ KB


In [9]:
df.isna().sum()

year                   0
network                0
average_audience       0
game_1_audience        1
game_2_audience        2
game_3_audience        1
game_4_audience        1
game_5_audience       10
game_6_audience       23
game_7_audience       36
total_games_played     0
winning_team           0
losing_team            0
losing_team_wins       0
dtype: int64

In [10]:
df

,year,network,average_audience,game_1_audience,game_2_audience,game_3_audience,game_4_audience,game_5_audience,game_6_audience,game_7_audience,total_games_played,winning_team,losing_team,losing_team_wins
0,2022,Fox,11762000,11475000,10789000,11162000,11809000,12786000,12549000,<NA>,6,Houston Astros,Philadelphia Phillies,2
1,2021,Fox,11744000,10811000,10280000,11232000,10511000,13644000,13986000,<NA>,6,Atlanta Braves,Houston Astros,2
2,2020,Fox,9785000,9195000,8950000,8156000,9332000,10059000,12627000,<NA>,6,Los Angeles Dodgers,Tampa Bay Rays,2
3,2019,Fox,13912000,12194000,11925000,12220000,10219000,11390000,16425000,23013000,7,Washington Nationals,Houston Astros,3
4,2018,Fox,14125000,13761000,13458000,13251000,13563000,17634000,<NA>,<NA>,5,Boston Red Sox,Los Angeles Dodgers,1
5,2017,Fox,18926000,14968000,15483000,15676000,15400000,18940000,22229000,28240000,7,Houston Astros,Los Angeles Dodgers,3
6,2016,Fox,22847000,19368000,17395000,19384000,16705000,23638000,23396000,40045000,7,Chicago Cubs,Cleveland Indians,3
7,2015,Fox,14533000,14944000,13722000,13205000,13587000,17206000,<NA>,<NA>,5,Kansas City Royals,New York Mets,1
8,2014,Fox,13930000,12191000,12917000,12133000,10742000,12635000,13372000,23517000,7,San Francisco Giants,Kansas City Royals,3
9,2013,Fox,14984000,14400000,13429000,12473000,15975000,14446000,19178000,<NA>,6,Boston Red Sox,St. Louis Cardinals,2


In [11]:
df.to_csv("world-series-ratings", index = False)